# TP2

In [ ]:
import re
import requests

from IPython.display import clear_output
from bs4 import BeautifulSoup

# Déclarations des fonctions

In [ ]:
# Enlever une certains caractères dans une chaine de caractère
def rchop(thestring, ending):
    if thestring.endswith(ending):
        return thestring[:-len(ending)]
    return thestring

# Fonction d'affichage
def info(*args, **kwargs):
    rulesArg = kwargs.get("rulesArg", False)
    wordArg = kwargs.get("wordArg", True)
    resultArg = kwargs.get("resultArg", True)
    
    if(rulesArg):
        print(f'\nLes règles :')
        pprint(rules)

    if(wordArg):
        print(f'\nLe mot : {testWord}')

    if(resultArg):
        print(f'\nLes dérivations :')
        for n in result.items():
            print(str(n).replace(',', ' ->'))

In [ ]:
# Déclarations des variables
nature = {'Verbe': 'V', 'Nom': 'N', 'Adjectif': 'ADJ', 'Nom propre': 'NP'}
rules = []
result = {}

# Lecture du mot
print(f'Entrer le mot : ')
testWord = input()

# Effacer les outputs
clear_output(wait = False)

# Lecture de la nature du mot
print(f'Veuillez choisir la nature du mot dans la liste ci-dessous : ')
for (key, value) in nature.items():
    print(f'\t{value} pour {key}')
natureWord = input()

# Effacer les outputs
clear_output(wait = False)

# Ouverture du fichier
file = open("rules.txt", "r")

# Lecture du fichier & enregistrement des règles
if file.mode == 'r':
    lines = file.readlines()
    for line in lines:
        rules.append(line.rstrip("\n").split("==>"))

# Fermeture du fichier
file.close()

# Génération des mots et leur nature
for rule in rules:
    ruleG = rule[0].split(":")
    ruleD = rule[1].split(":")
    match = re.match(".*" + ruleG[1], testWord)
    if match and ruleG[0] == natureWord:
        if ruleG[1] != "":
            generatedWord = rchop(testWord, ruleG[1]) + ruleD[1]
            result[generatedWord] = ruleD[0]
        else:
            generatedWord = testWord + ruleD[1]
            result[generatedWord] = ruleD[0]

# Fonction d'affichage (avec possibilité d'afficher que ce qui nous intéresse)
info(rulesArg = False, wordArg = False, resultArg = False)

In [ ]:
final = []
for word in result:
    # Envoie de la requete à jdm
    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(word) + "&rel="
    
    # Reponse de la requete
    response = requests.get(url)
    
    # bs4
    soup = BeautifulSoup(response.text, features = 'lxml')
    
    # Chercher la balise "jdm-warning"
    mydivs = soup.findAll("div", {"class": "jdm-warning"})
    
    if(mydivs): # if jdm-warning existe (mot invalide)
        print(f'le mot {word} n\'existe pas\n')
    else: # else (mot valide)
        print(f'le mot {word} existe\n')    
        final.append(word)
    
if(final):
    print('Les mots valides sont :')
    for f in final:
        print(f'\t{f} : {result[f]}')
else:
    print('Aucun mot n\'est valide')

In [ ]:
print(testWord)
    
relation_map = {'AGENT': '13', 'LIEU' : '15', 'N': '4'}

for f in final:
    print(f'\t{f} : {result[f]}')
    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(testWord) + "&rel=" + str(relation_map[result[f]])
    print(url)
    
    # On cherche si la relation existe dans la balise code